# WIP

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchmetrics import Accuracy

from pyroml import PyroModule, Trainer, Stage
from pyroml.models import Backbone, TimmBackbone
from pyroml.models.utils import get_features

In [7]:
class PyroTeacher(PyroModule):
    def __init__(self, backbone: TimmBackbone):
        super().__init__()
        self.backbone = backbone
        self.head = nn.Linear(self.last_dim, 102)
        self.acc = Accuracy(task="multiclass", num_classes=102, average="macro")
        self.loss = nn.CrossEntropyLoss()

    def train(self, mode=True):
        self.backbone.eval()
        self.head.train(mode)
        return self

    @property
    def last_dim(self) -> int:
        return self.backbone.last_dim[0]

    def forward(self, x):
        with torch.no_grad():
            x = self.backbone(x)
        x = self.head(x)
        return x

    def step(self, batch, stage):
        x, y = batch["image"], batch["label"]
        preds = self(x)
        loss = self.loss(preds, y)
        self.log(loss=loss.item(), acc=self.acc(preds, y))
        return loss

In [8]:
from pyroml.models.utils import freeze_module


backbone = Backbone.load("resnet50")
backbone.eval()
freeze_module(backbone)
teacher = PyroTeacher(backbone)
teacher

Backbone has 23,508,032 params


PyroTeacher(
  (backbone): TimmBackbone(
    (model): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act1): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (drop_block): Identity()
          (act2): ReLU(inplace=True)
          (aa): Identity()
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1

In [5]:
teacher.last_dim

512

In [ ]:
from torchvision.tv_tensors import Image
from pyroml.template.flowers102.dataset import Flowers102Dataset
from torchvision.transforms import v2

img_size = 224


class ToBoundedFloat(nn.Module):
    def forward(self, img: Image):
        return (img / 255.0).float()


transform = v2.Compose(
    [
        v2.ToImage(),
        ToBoundedFloat(),
        v2.Resize((img_size, img_size)),
        v2.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ]
)
tr_ds = Flowers102Dataset(split="train", transform=transform)
te_ds = Flowers102Dataset(split="test", transform=transform)

(
    len(tr_ds),
    len(te_ds),
    tr_ds[0].keys(),
    tr_ds[0]["image"].shape,
)


(7169, 1020, dict_keys(['image', 'label']), torch.Size([3, 224, 224]))

In [ ]:
from pyroml.callbacks.progress.tqdm_progress import TQDMProgress

trainer = Trainer(
    lr=0.05,
    max_epochs=16,
    batch_size=17,
    evaluate_on=False,
    pin_memory=False,
    dtype=torch.float32,
    callbacks=[TQDMProgress()],
)

In [ ]:
tr_tracker = trainer.fit(teacher, tr_dataset=tr_ds, ev_dataset=te_ds)
tr_tracker.get_last_epoch_metrics()

In [ ]:
backbone.model.fc = nn.Identity()
teacher = PyroTeacher(backbone)
teacher.backbone._compute_last_dim(image_size=(3, img_size, img_size))
teacher.last_dim

## Retrieve teacher features

In [ ]:
# Preccompute teacher features to avoid wasting ressources during student training
tr_feats = get_features(teacher, tr_ds, dtype=torch.bfloat16, batch_size=16)
te_feats = get_features(teacher, te_ds, dtype=torch.bfloat16, batch_size=16)

In [ ]:
class Flowers102FeatsDataset(torch.utils.data.Dataset):
    """
    Uses the precomputed features from SmolVLM vision model and the dataset labels
    """

    def __init__(self, feats: torch.Tensor, dataset: Flowers102Dataset):
        assert len(feats) == len(dataset)
        self.feats = feats
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        item["feat"] = self.feats[idx]
        return item

In [ ]:
tr_feats_ds = Flowers102FeatsDataset(feats=tr_feats, dataset=tr_ds)
te_feats_ds = Flowers102FeatsDataset(feats=te_feats, dataset=te_ds)

## Distill the teacher

In [ ]:
class StudentModule(PyroModule):
    def __init__(
        self,
        teacher: TimmBackbone,
        temperature: float = 2,
        alpha: float = 1,
        num_classes: int = 102,
    ):
        super().__init__()
        self.teacher = teacher
        self.temperature = temperature
        self.alpha = alpha
        self.student = nn.Sequential(
            nn.Conv2d(3, 6, 3),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(6, 24, 3),
            nn.BatchNorm2d(24),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(24, 48, 3),
            nn.BatchNorm2d(48),
            nn.ReLU(),
            nn.MaxPool2d(4),
            nn.Conv2d(48, 96, 3),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.Conv2d(96, 8, 1),
            nn.Flatten(),
            nn.Linear(968, teacher.last_dim),
        )
        self.student_head = nn.Sequential(
            nn.ReLU(), nn.Linear(teacher.last_dim, num_classes)
        )

        self.soft_loss = nn.KLDivLoss(reduction="batchmean")
        self.hard_loss = nn.CrossEntropyLoss()

        metrics_kwargs = dict(
            task="multiclass", num_classes=num_classes, average="macro"
        )
        self.acc = Accuracy(**metrics_kwargs)
        self.acc_5 = Accuracy(**metrics_kwargs, top_k=5)

    def configure_optimizers(self, loop):
        self.optimizer = torch.optim.Adam(self.student.parameters(), lr=self.trainer.lr)

    def train(self, mode=True):
        # Prevent the teacher to switch to train mode
        return self.student.train(mode=mode)

    def forward(self, x) -> dict[str, torch.Tensor]:
        logits = self.student(x)
        preds = self.student_head(logits)
        return {"logits": logits, "preds": preds}

    def step(self, batch, stage) -> torch.Tensor:
        x, teacher_logits, label = batch["image"], batch["feat"], batch["label"]

        student_x = self(x)
        student_logits = student_x["logits"]
        student_preds = student_x["preds"]

        soft_teacher = F.softmax(teacher_logits / self.temperature, dim=-1)
        soft_student = F.log_softmax(student_logits / self.temperature, dim=-1)
        soft_loss = self.soft_loss(soft_student, soft_teacher) * self.temperature**2

        hard_loss = self.hard_loss(student_preds, label)

        loss = hard_loss + self.alpha * soft_loss

        preds = torch.softmax(student_preds, dim=-1)
        self.log(
            loss=loss.item(),
            soft=soft_loss.item(),
            hard=hard_loss.item(),
            acc=self.acc(preds, label),
            acc_5=self.acc_5(preds, label),
        )

        if stage == Stage.TRAIN:
            return loss

        return preds


In [ ]:
student = StudentModule(teacher=teacher)
student

In [ ]:
from pyroml.models.utils import num_params


t_params = num_params(teacher)
s_params = num_params(student.student)

f"Teacher = {t_params:,}, Student = {s_params:,}, Ratio = {100 * s_params / t_params:.3f}%"

In [ ]:
from pyroml.callbacks.progress.tqdm_progress import TQDMProgress


trainer = Trainer(
    lr=0.001,
    max_epochs=16,
    batch_size=17,
    evaluate_on=False,
    dtype=torch.bfloat16,
    callbacks=[TQDMProgress()],
)
tr_tracker = trainer.fit(student, tr_dataset=tr_feats_ds)

In [ ]:
tr_tracker.plot(epoch=True)

In [ ]:
import rich

te_tracker = trainer.evaluate(model=student, dataset=te_feats_ds)

rich.print(te_tracker.get_last_epoch_metrics())